In [1]:
print("Setting everything up!")
import warnings
warnings.filterwarnings('ignore')

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, CustomJS, FixedTicker
import bokeh.palettes as pal
from bokeh.layouts import layout, Spacer, gridplot

Setting everything up!


In [3]:
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output

In [6]:
import numpy as np
from bokeh.charts import Bar, Histogram

/home/sychen23/.local/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [7]:
from analysis import classify_image, glimpses, read_n, classify_imgs2

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', 'true', 'true', 'true', 'true', 'true', 'true', 'model_runs/regimen/classify_log.csv', 'model_runs/regimen/classifymodel_0.ckpt', 'model_runs/regimen/classifymodel_', 'model_runs/regimen/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
analysis.py


In [8]:
# import numpy as np
# import scipy.special

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from scipy import linspace
from scipy import pi,sqrt,exp
from scipy.special import erf
import scipy

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show

In [9]:
clear_output()
b2 = Button(description="Click to Start", icon="arrow", width=400)

dropdown2 = Dropdown(options=['0', '1000', '2000', '3000', '4000', '5000', '10000', '20000', '30000', '40000', '50000',
                             '60000', '70000', '80000', '90000', '100000', '110000', '120000', '130000', '140000', '150000',
                             '160000', '170000', '180000', '190000', '200000', '250000', '300000', '400000', '500000',
                             '600000', '700000', '800000', '900000', '910000', '920000', '1000000', '1100000', '1200000', '1300000', '1400000', '1500000'],
                    value='10000', 
                    description='Iteration:'
)

data = None
    
    
def get_pdf(sigma, x, mu):
    """Calculate the pdf."""
    
#     pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
    pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    return pdf


def get_cdf(sigma, x, mu, a):
    """Calculate the cdf."""
    
#     cdf = (1 + erf(a*x / sqrt(2))) / 2 
    cdf = (1 + scipy.special.erf((a*x - mu) / np.sqrt(2*sigma**2))) / 2
    return cdf


def get_p(pdf, cdf):
    """Calculate p to create skew curve."""
    
    p = 2 / pdf * cdf / 100000
    return p

    
def curve(sigma, x, mu, plot, label="", color="gray"):
    """Add normal distribution curve to plot."""
    
    pdf = get_pdf(sigma, x, mu)
    plot.line(x, pdf, line_color=color, line_width=8, alpha=1, legend=label)
    
    
def skew_curve(sigma, x, mu, a, plot):
    """Add a skewed curve to the plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))

    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(9-x, p / f(max_x), line_color="blue", line_width=8, alpha=0.4)
    
    
def combined_curve(sigma, x, mu, a, plot):
    """Add curve combining skew and normal distribution to plot."""
    
    p = get_p(get_pdf(sigma, x, mu), get_cdf(sigma, x, mu, a))
    
    def f(x): return 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
    max_x = scipy.optimize.fmin(lambda x: -f(x), a)

    def f2(x): return (1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)) + p / f(max_x))
    max_x2 = scipy.optimize.fmin(lambda x: -f(x), a)

    plot.line(x, (((1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2)))
            + p / f(max_x))/ f2(max_x2)), line_color="purple", line_width=2, alpha=1)
    

def update_curves():
    clear_output()
    global data
    num_imgs = 10000
    print("number of images: %d" % num_imgs)
    imgs_data = classify_imgs2(int(dropdown2.value), True, num_imgs)
    
#     num_blobs = randint(0, 9)
    max_blobs = 9
    min_blobs = 0
    
    curves = list()
    dark = "#111111"
    light = "#DDDDDD"
    p2 = figure(title="Blob Number Classification Probabilities Distributions", y_range=(0, 1), tools="save", background_fill_color=dark)
    p2.border_fill_color = dark
    p2.title.text_color = light
    p2.xaxis.axis_label_text_color = light
    p2.yaxis.axis_label_text_color = light
    p2.xaxis.axis_line_color = light
    p2.yaxis.axis_line_color = light
    p2.xaxis.major_label_text_color = light
    p2.yaxis.major_label_text_color = light
    p2.xaxis.major_tick_line_color = light
    p2.xaxis.minor_tick_line_color = light
    p2.yaxis.major_tick_line_color = light
    p2.yaxis.minor_tick_line_color = light

    for num_blobs in range(min_blobs, max_blobs + 1):
        print("num_blobs: ", num_blobs)
        

        p1 = figure(title="Blob Number Classification Probabilities Distribution for %d Blobs" % num_blobs, y_range=(0, 1), tools="save",
                    background_fill_color=dark)
        p1.border_fill_color = dark
        p1.title.text_color = light
        p1.xaxis.axis_label_text_color = light
        p1.yaxis.axis_label_text_color = light
        p1.xaxis.axis_line_color = light
        p1.yaxis.axis_line_color = light
        p1.xaxis.major_label_text_color = light
        p1.yaxis.major_label_text_color = light
        p1.xaxis.major_tick_line_color = light
        p1.xaxis.minor_tick_line_color = light
        p1.yaxis.major_tick_line_color = light
        p1.yaxis.minor_tick_line_color = light



        m = 0.1
        z_size = 10

        new_hist = np.zeros(z_size)
        choice_hist = np.zeros(z_size)
        value_counts = np.zeros(z_size)
        values_sum = 0
        sqr_sum = 0
        num_imgs_with_num_blobs = 0

        for idx, data in enumerate(imgs_data):

            if data["label"][num_blobs] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1
                
                max_glimpse = 2
                min_glimpse = 0
                glimpses = 5#max_glimpse - min_glimpse + 1
                
                for i in [0, 1, 2, 3, 4]:#range(min_glimpse, max_glimpse + 1):
                    # Histogram of softmaxes
                    new_hist += data["classifications"][i][0] / glimpses
                    
                    # Histogram of choices
                    choice = np.argmax(data["classifications"][i][0])
                    choice_list = [0] * z_size
                    choice_list[choice] = 1 / glimpses
                    choice_hist += choice_list
                    
#                 glimpse = 1
        
#                 new_hist += data["classifications"][glimpse][0]
        
#                 choice = np.argmax(data["classifications"][glimpse][0])
#                 choice_list = [0] * z_size
#                 choice_list[choice] = 1
#                 choice_hist += choice_list
                
        print("num_imgs_with_num_blobs: ", num_imgs_with_num_blobs)
        
        new_hist = new_hist / num_imgs_with_num_blobs
        print("new_hist: ", new_hist)
        choice_hist = choice_hist / num_imgs_with_num_blobs

        x = np.linspace(-2, 11.0, 1000)
        source = ColumnDataSource(data=dict(color=["red"] * z_size, top=new_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.55, right=np.arange(1, z_size + 1) - m - 0.55))
        source2 = ColumnDataSource(data=dict(color=["yellow"] * z_size, top=choice_hist, bottom=np.zeros(z_size), left=np.arange(z_size) + m - 0.45, right=np.arange(1, z_size + 1) - m - 0.45))
        source.data["color"][num_blobs] = "lime"
        p1.quad('left', 'right', 'top', 'bottom', source=source, color="color", alpha=1)
        p1.quad('left', 'right', 'top', 'bottom', source=source2, color="color", alpha=0.5)


        # FORMAT PLOT ##############################

        p1.xaxis.axis_label = 'Number of Blobs'
        p1.yaxis.axis_label = 'Classification Probability'
        p1.xaxis[0].ticker=FixedTicker(ticks=np.arange(z_size))
        
        
        # PLOT CURVES #############################
        
        # Find the mean
        for j in range(z_size):
            values_sum += j * new_hist[j]
        mu = values_sum
        print("mu: ", mu)

        # Find the standard deviation
        for k in range(z_size):
            sqr_sum += new_hist[k] * ((k - mu) ** 2)
        sigma = np.sqrt(sqr_sum)
        
        curves.append((sigma * 2, x, mu, p2, str(num_blobs),
                       "#" + str(randint(2, 9)) + str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))+ str(randint(2, 9))))
        
        curve(sigma, x, mu, p1) # plot gaussian curve
#         a = np.argmax(new_hist)
#         skew_curve(sigma, x, mu, a, p1)
#         combined_curve(sigma, x, mu, a, p1)

        show(gridplot(p1, ncols=2, plot_width=500, plot_height=400, toolbar_location=None))
    
    for stats in curves:
        curve(*stats)
    show(p2)
    

def on_click2(b2, new_image=True):
    """Load new random image after button is clicked."""
    
    b2.description = "Loading..."
    update_curves()
    b2.description = "Next (Random) Image"

b2.on_click(on_click2)


def on_change2(change):
    """Change the iteration number to new dropdown selection."""
    
    if change['type'] == 'change' and change['name'] == 'value':
        on_click(b2, new_image=False)
        

dropdown2.observe(on_change2)
display(HBox([b2, dropdown2]))

number of images: 10000
INFO:tensorflow:Restoring parameters from model_runs/max_delta/classifymodel_10000.ckpt


NotFoundError: Key optimizer1/position/b/Adam not found in checkpoint
	 [[Node: save/RestoreV2_20 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_20/tensor_names, save/RestoreV2_20/shape_and_slices)]]
	 [[Node: save/RestoreV2_235/_453 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_3106_save/RestoreV2_235", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_20', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/co